# BSDS4 Deep Learning Elective
2022 DS Elective 4 <br>
University of Science and Technology of the Philippines <br>
Instructor: Jhun Brian M. Andam <br>
Credits: Mr. Romen Samuel Wabina MSc.

## PyTorch - Linear Regression

- [WEIDMAN] Ch7
- https://pytorch.org/tutorials/

In [1]:
import torch
import numpy as np
import sys

In [2]:
torch.__version__

'2.0.1+cu117'

In [3]:
#We can check whether we have gpu
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print("Device: ", device)

Device:  cpu


Let's have linear regression as a case study to study the different components of PyTorch.  These are the following components we will be covering:

1. Specifying input and target
2. Dataset and DataLoader
3. `nn.Linear` (Dense)
4. Define loss function
5. Define optimizer function
6. Train the model

Consider this data:

<img src = "figures/japan.png" width="400">

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

$$\text{yield}_\text{apple}  = w_{11} * \text{temp} + w_{12} * \text{rainfall} + w_{13} * \text{humidity} + b_{1}$$

$$\text{yield}_\text{orange} = w_{21} * \text{temp} + w_{22} * \text{rainfall} + w_{23} * \text{humidity} + b_{2}$$

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

<img src = "figures/japan2.png" width="400">

The learning part of linear regression is to figure out a set of weights <code>w11, w12,... w23, b1 \& b2</code> using gradient descent

#### 1. Specifiying input and target

In [4]:
# Input (temp, rainfall, humidity)
x_train = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
y_train = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

In [5]:
inputs = torch.from_numpy(x_train)
targets = torch.from_numpy(y_train)
print(inputs.size())
print(targets.size())

torch.Size([15, 3])
torch.Size([15, 2])


#### 2. Dataset and DataLoader

We'll create a <code>TensorDataset</code>, which allows access to rows from inputs and targets as tuples, and if we want to use <code>DataLoader</code> (will talk shortly) from numpy array, we have to first make <code>TensorDataset</code>.

In [6]:
from torch.utils.data import TensorDataset

In [7]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

We'll now create a <code>DataLoader</code>, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [8]:
from torch.utils.data import DataLoader

In [9]:
# Define data loader
batch_size = 3
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a for-in loop. Let's look at an example

In [10]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.]])
tensor([[ 22.,  37.],
        [ 81., 101.],
        [103., 119.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

#### 3. Define some layer - nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the <code>nn.Linear</code> class from PyTorch, which does it automatically.

In [11]:
import torch.nn as nn

# Define model
model = nn.Linear(3, 2)  #nn.Linear assume this shape (in_features, out_features)
print(model.weight)
print(model.weight.size()) # (out_features, in_features)
print(model.bias)
print(model.bias.size()) #(out_features)

Parameter containing:
tensor([[-0.1799,  0.4061, -0.5485],
        [-0.0350, -0.1383, -0.1410]], requires_grad=True)
torch.Size([2, 3])
Parameter containing:
tensor([-0.5605,  0.5771], requires_grad=True)
torch.Size([2])


In fact, our model is simply a function that performs a matrix multiplication of the <code>inputs</code> and the weights <code>w</code> and adds the bias <code>b</code> (for each observation)

<img src = "figures/dot.png" width="400">

PyTorch models also have a helpful <code>.parameters</code> method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [12]:
# Parameters
list(model.parameters())  #model.param returns a generator

[Parameter containing:
 tensor([[-0.1799,  0.4061, -0.5485],
         [-0.0350, -0.1383, -0.1410]], requires_grad=True),
 Parameter containing:
 tensor([-0.5605,  0.5771], requires_grad=True)]

In [13]:
#we can print the complexity by the number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

8


We can use the <code>model(tensor)</code> API to perform a forward-pass that generate predictions

In [14]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-10.0708, -17.3106],
        [-16.2995, -23.8073],
        [  6.3941, -29.1850],
        [-21.7457, -14.1589],
        [-12.3826, -24.9904],
        [-10.0708, -17.3106],
        [-16.2995, -23.8073],
        [  6.3941, -29.1850],
        [-21.7457, -14.1589],
        [-12.3826, -24.9904],
        [-10.0708, -17.3106],
        [-16.2995, -23.8073],
        [  6.3941, -29.1850],
        [-21.7457, -14.1589],
        [-12.3826, -24.9904]], grad_fn=<AddmmBackward0>)

#### 4. Define loss function

The <code>nn</code> module contains a lot of useful loss function like this:

In [15]:
criterion_mse = nn.MSELoss()
criterion_softmax_cross_entropy_loss = nn.CrossEntropyLoss()

In [16]:
mse = criterion_mse(preds, targets)
print(mse)
print(mse.item())  ##print out the loss number

tensor(11459.3887, grad_fn=<MseLossBackward0>)
11459.388671875


#### 5. Define the optimizer

We use <code>optim.SGD</code> to perform stochastic gradient descent where samples are selected in batches (often with random shuffling) instead of as a single group.  Note that <code>model.parameters()</code> is passed as an argument to <code>optim.SGD</code>.

In [17]:
# Define optimizer
#momentum update the weight based on past gradients also, which will be useful for getting out of local max/min
#If our momentum parameter was $0.9$, we would get our current grad + the multiplication of the gradient 
#from one time step ago by $0.9$, the one from two time steps ago by $0.9^2 = 0.81$, etc.

opt = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

#### 6. Training - putting everything together

In [24]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            xb.to(device) #move them to gpu if possible, if not, it will be cpu
            yb.to(device)
                    
            # 1. Predict
            pred = model(xb)
                      
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Calculate gradient
            opt.zero_grad()  #if not, the gradients will accumulate
            loss.backward()
            
            # Print out the gradients.
            print ('dL/dw: ', model.weight.grad) 
            print ('dL/db: ', model.bias.grad)
            
            # 4. Update parameters using gradients
            opt.step()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            sys.stdout.write("\rEpoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))

In [25]:
#train for 100 epochs
fit(100, model, criterion_mse, opt, train_dl)

dL/dw:  tensor([[-114.9531,  -49.6868,  -42.2578],
        [ -54.8290,  -23.7081,  -20.1598]])
dL/db:  tensor([-1.1366, -0.5422])
dL/dw:  tensor([[-54.7069, -87.3302, -63.7267],
        [-46.5612, -48.7738, -35.4951]])
dL/db:  tensor([-0.8855, -0.5426])
dL/dw:  tensor([[131.0922, 143.7197,  71.5849],
        [  2.2614,  55.3385,  15.5958]])
dL/db:  tensor([1.4189, 0.1043])
dL/dw:  tensor([[120.0732, 112.4399,  72.7369],
        [ 37.9047,  46.1896,  32.5710]])
dL/db:  tensor([1.6486, 0.5384])
dL/dw:  tensor([[ -89.4453, -137.8302,  -59.6261],
        [ -65.8574,  -59.2965,  -46.5978]])
dL/db:  tensor([-1.0282, -0.7198])
dL/dw:  tensor([[-112.8658, -165.8326,  -75.2382],
        [  16.9010,   34.8446,   10.4938]])
dL/db:  tensor([-1.3024,  0.2070])
dL/dw:  tensor([[60.1773, 52.9520, 38.4780],
        [36.9622, 58.1428, 42.4248]])
dL/db:  tensor([0.6180, 0.5912])
dL/dw:  tensor([[ 150.0963,   96.2875,   69.5007],
        [-222.3177, -102.5595,  -84.6747]])
dL/db:  tensor([ 1.7306, -2.248

In [20]:
# Generate predictions
preds = model(inputs)
loss = criterion_mse(preds, targets)
print(loss.item())

1.927969217300415


In [28]:
model.weight.grad

tensor([[-567.3395, -728.3865, -338.7109],
        [ 235.6568,  318.6789,  145.0725]])

In [23]:
y_train

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.],
       [ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.],
       [ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)